In [2]:
import krakenex 
from pykrakenapi import KrakenAPI
import sys
import time
import urllib.request
import json
import requests
import pandas as pd
import numpy as np 

In [5]:
api = krakenex.API()
k = KrakenAPI(api)

In [6]:
ohlc = k.get_ohlc_data("BTCUSD")
print(ohlc)

attempt: 000 | 502 Server Error: Bad Gateway for url: https://api.kraken.com/0/public/OHLC
attempt: 001 | 502 Server Error: Bad Gateway for url: https://api.kraken.com/0/public/OHLC
attempt: 002 | 502 Server Error: Bad Gateway for url: https://api.kraken.com/0/public/OHLC
(                           time     open     high      low    close     vwap  \
dtime                                                                          
2021-06-05 18:31:00  1622917860  36151.6  36151.6  36151.6  36151.6      0.0   
2021-06-05 18:30:00  1622917800  36151.6  36151.6  36151.6  36151.6      0.0   
2021-06-05 18:29:00  1622917740  36151.6  36151.6  36151.6  36151.6      0.0   
2021-06-05 18:28:00  1622917680  36151.6  36151.6  36151.6  36151.6      0.0   
2021-06-05 18:27:00  1622917620  36151.6  36151.6  36151.6  36151.6      0.0   
...                         ...      ...      ...      ...      ...      ...   
2021-06-05 06:36:00  1622874960  37733.5  37733.5  37701.1  37701.1  37733.2   
2021-0

In [7]:
def fetch_OHLC_data(symbol, timeframe):
    pair_split = symbol.split('/')
    symbol = pair_split[0] + pair_split[1]
    url = f'https://api.kraken.com/0/public/OHLC?pair={symbol}&interval={timeframe}'
    response = requests.get(url)
    if response.status_code == 200: 
        j = json.loads(response.text)
        result = j['result']
        keys = []
        for item in result:
            keys.append(item)
        if keys[0] != 'last':
            data = pd.DataFrame(result[keys[0]],
                                columns=['unix', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'tradecount'])
        else:
            data = pd.DataFrame(result[keys[1]],
                                columns=['unix', 'open', 'high', 'low', 'close', 'vwap', 'volume', 'tradecount'])

        data['date'] = pd.to_datetime(data['unix'], unit='s')
        data['volume_from'] = data['volume'].astype(float) * data['close'].astype(float)
    return data

In [25]:
btc = fetch_OHLC_data(symbol="BTC/USD", timeframe="1440")
btc = btc.drop(columns=["unix", "open", "high", "low", "vwap", "volume", "tradecount", "volume_from"])
btc = btc.set_index("date")
btc

,close
date,
2019-06-17,9335.0
2019-06-18,9080.5
2019-06-19,9277.1
2019-06-20,9534.3
2019-06-21,10219.2
...,...
2021-06-01,36676.3
2021-06-02,37565.3
2021-06-03,39196.6


In [26]:
def generate_signals(btc):
    """Generates trading signals for a given dataset."""
    # Grab just the `date` and `close` from the IEX dataset
    signals_df = btc.copy()
    # Set the short window and long windows
    short_window = 50
    long_window = 100
    # Generate the short and long moving averages (50 and 100 days, respectively)
    signals_df["SMA50"] = signals_df["close"].rolling(window=short_window).mean()
    signals_df["SMA100"] = signals_df["close"].rolling(window=long_window).mean()
    signals_df["Signal"] = 0.0
    # Generate the trading signal 0 or 1,
    # where 0 is when the SMA50 is under the SMA100, and
    # where 1 is when the SMA50 is higher (or crosses over) the SMA100
    signals_df["Signal"][short_window:] = np.where(
        signals_df["SMA50"][short_window:] > signals_df["SMA100"][short_window:],
        1.0,
        0.0,
    )
    # Calculate the points in time at which a position should be taken, 1 or -1
    signals_df["Entry/Exit"] = signals_df["Signal"].diff()
    return signals_df

In [28]:
mov_avg = generate_signals(btc)
mov_avg.tail()

C:\ProgramData\Anaconda3\envs\dev\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
2021-06-01,36676.3,49756.730,52276.144,0.0,0.0
2021-06-02,37565.3,49236.292,52110.249,0.0,0.0
2021-06-03,39196.6,48760.844,52013.217,0.0,0.0
2021-06-04,36847.7,48232.938,51884.417,0.0,0.0
2021-06-05,36151.6,47726.870,51775.318,0.0,0.0


In [30]:
bollinger_window = 20

# Calculate rolling mean and standard deviation
all_sig = mov_avg.copy()
all_sig['bollinger_mid_band'] = mov_avg['close'].rolling(window=bollinger_window).mean()
all_sig['bollinger_std'] = mov_avg['close'].rolling(window=20).std()

# Calculate upper and lowers bands of bollinger band
all_sig['bollinger_upper_band']  = all_sig['bollinger_mid_band'] + (all_sig['bollinger_std'] * 1)
all_sig['bollinger_lower_band']  = all_sig['bollinger_mid_band'] - (all_sig['bollinger_std'] * 1)

all_sig

,close,SMA50,SMA100,Signal,Entry/Exit,bollinger_mid_band,bollinger_std,bollinger_upper_band,bollinger_lower_band
date,,,,,,,,,
2019-06-17,9335.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2019-06-18,9080.5,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2019-06-19,9277.1,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2019-06-20,9534.3,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
2019-06-21,10219.2,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
2021-06-01,36676.3,49756.730,52276.144,0.0,0.0,40056.350,4834.786318,44891.136318,35221.563682
2021-06-02,37565.3,49236.292,52110.249,0.0,0.0,39450.855,4294.908779,43745.763779,35155.946221
2021-06-03,39196.6,48760.844,52013.217,0.0,0.0,38915.735,3521.755172,42437.490172,35393.979828
